# ketos jupyter notebook machine learning example

this is an example for a ketos jupyter notebook.

to deploy the notebook in order to allow for external predictions just do the following:
1. tag the cells that you would like to be executed for your prediction with the keyword "ketos_predict" in the first line of each cell as a comment.

2. tag the cells which produce the output for the prediction with the keyword "ketos_predict_output", please note that this cell will be executed and the first line of the output will be used as return value for the prediction


In [1]:
install.packages(c('caret', 'plyr', 'e1071'))

Updating HTML index of packages in '.Library'
Warning message in file.create(f.tg):
“cannot create file '/usr/share/doc/R/html/packages.html', reason 'Permission denied'”Warning message in make.packages.html(.Library):
“cannot update HTML package index”

In [9]:
# ketos_train
genPrep = function(){  
  library(caret)
}

splitData = function(seed, data){
  # randomly splits data into training (0.8) and test (0.2) data
  
  set.seed(seed)
  training_index = createDataPartition(data[[ncol(data)]], p = 0.8, list = F)
  training = data[training_index,]
  test = data[-training_index,]
  
  return(list(training=training,test=test))
}

In [10]:
# ketos_train
ketos_train = function(dataset = NULL, outcomeVar = NULL, metric = "Accuracy", control = NULL){
  # trains model
  form = as.formula(paste(c(outcomeVar, "~."), collapse=" "))
  
  print("start train GLMNET")
  set.seed(seed)
  trained_model = caret::train(form, data=dataset, method="glmnet", metric=metric, preProc=c("center", "scale"), trControl=control)

  return(trained_model)
  
}

In [11]:
# ketos_train
ketos_data_file = "iris_data_ml_omop_final.csv"

ketos_data_train = read.csv(ketos_data_file, header = TRUE, sep = ";", quote = "\"",dec = ".", fill = TRUE, comment.char = "")
genPrep()

seed = 7
splitData = splitData(seed, ketos_data_train)
trainingData = splitData$training
testData = splitData$test
fileTrain = "training.csv"
fileTest  = "test.csv"

write.table(trainingData, fileTrain, sep = ";", quote = TRUE, dec = ".", row.names = FALSE, append = FALSE)
write.table(testData, fileTest, sep = ";", quote = TRUE, dec = ".", row.names = FALSE, append = FALSE)

control = trainControl(method="repeatedcv", number=10, repeats=10, sampling="down", savePredictions="all")
outcomeVar = "human.body.structure"
ketos_model = ketos_train(trainingData, outcomeVar,"Accuracy",control)
modeltoSave = 'ketos_model'
save(ketos_model,file = modeltoSave)

[1] "start train GLMNET"


In [1]:
#ketos_predict, ketos_predict_output
ketos_predict = function(){
    ketos_predict_data = ("tmpDummyUrl-will be set by the system automatically insert your tmp url here to get a data example")
    load("ketos_model")
    predict_data = read.csv('test.csv', header = TRUE, sep = ";", quote = "\"",dec = ".", fill = TRUE, comment.char = "")
    
    prediction = predict(ketos_model, predict_data)
    return (prediction)
}

print(as.character(ketos_predict()))

Warning message in as.POSIXlt.POSIXct(Sys.time()):
“unknown timezone 'zone/tz/2017c.1.0/zoneinfo/Europe/Berlin'”

[1] "bad body structure"
